In [11]:
import numpy as np
import scipy
import scipy.io
import argparse
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import pandas as pd


ld = float(500000)

HITK = 2

def normalizeFeature(x):
    # x = d x N dims (d: feature dimension, N: the number of features)
    x = x + 1e-10 # for avoid RuntimeWarning: invalid value encountered in divide
    feature_norm = np.sum(x**2, axis=1)**0.5 # l2-norm
    feat = x / feature_norm[:, np.newaxis]
    return feat

def SAE(x, s, ld):
    # SAE is Semantic Autoencoder
    # INPUTS:
    # 	x: d x N data matrix
    #	s: k x N semantic matrix
    #	ld: lambda for regularization parameter
    #
    # OUTPUT:
    #	w: kxd projection matrix

    A = np.dot(s, s.transpose())
    B = ld * np.dot(x, x.transpose())
    C = (1+ld) * np.dot(s, x.transpose())
    w = scipy.linalg.solve_sylvester(A,B,C)
    print(w.shape)
    return w

def distCosine(x, y):
    xx = np.sum(x**2, axis=1)**0.5
    x = x / xx[:, np.newaxis]
    yy = np.sum(y**2, axis=1)**0.5
    y = y / yy[:, np.newaxis]
    dist = 1 - np.dot(x, y.transpose())
    return dist



def zsl_acc(semantic_predicted, semantic_gt):
    # zsl_acc calculates zero-shot classification accruacy
    #
    # INPUTS:
    #	semantic_prediced: predicted semantic labels
    # 	semantic_gt: ground truth semantic labels
    # 	opts: other parameters
    #
    # OUTPUT:
    # 	zsl_accuracy: zero-shot classification accuracy (per-sample)

    dist = 1 - distCosine(semantic_predicted, normalizeFeature(semantic_gt.transpose()).transpose())
    y_hit_k = np.zeros((dist.shape[0], HITK))
    for idx in range(0, dist.shape[0]):
        sorted_id = sorted(range(len(dist[idx,:])), key=lambda k: dist[idx,:][k], reverse=True)
        y_hit_k[idx,:] = test_classes_id[sorted_id[0:HITK]]
    #sns.set()
    #C2 = confusion_matrix(test_labels, y_hit_k, labels=[5, 13, 14, 17, 23, 24, 33, 38, 41, 47])
    '''
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(C2)
    plt.title('Confusion matrix for novel class')
    fig.colorbar(cax)
    ax.set_xticklabels([''] + labels)
    ax.set_yticklabels([''] + labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()
    '''
    #ax = plt.subplot()
    #sns.heatmap(C2,annot=True, ax = ax)
    #labels=[5, 13, 14, 17, 23, 24, 33, 38, 41, 47]
    #cmatrix = np.array(confusion_matrix(opts.test_labels, y_hit_k, normalize='true'))
    #disp = ConfusionMatrixDisplay(cmatrix, labels)
    #fig, ax = plt.subplots(figsize=(10, 10))
    #ax.set_title('Confusion Matrix for novel class')   
    #disp.plot(xticks_rotation=-45.0, ax=ax, cmap='Blues')
    #print(C2)
    n = 0
    for idx in range(0, dist.shape[0]):
        if test_labels[idx] in y_hit_k[idx,:]:
            n = n + 1
    zsl_accuracy = float(n) / dist.shape[0] * 100
    return zsl_accuracy, y_hit_k


# for AwA dataset: Perfectly works.
'''
AWA data details:
X_tr: (24295, 1024)
X_te: (6108, 1024)
S_tr: (24295, 85)
test_labels: (6180,)
testclasses_id:(10,)
S_te_gt: (10, 85)
'''

'''
# AWA
opts = parse_args()
awa = scipy.io.loadmat('../data/awa_demo_data.mat')
train_data = awa['X_tr']
test_data = awa['X_te']
train_class_attributes_labels_continuous_allset = awa['S_tr']
opts.test_labels = awa['test_labels']
opts.test_classes_id = awa['testclasses_id']
test_class_attributes_labels_continuous = awa['S_te_gt']
'''

# AWA2_test_attributlabel: (6985, 85)
# AWA2_test_continuous_01_attributelabel: (6985, 85)
# AWA2_testlabel: (6985,) -> test_labels
# AWA2_train_continuous_01_attributelabel: (30337, 85) -> S_tr
# AWA2_trainlabel: (30337,)
# resnet101_testfeatures: (6985, 2048) -> X_te
# resnet101_trainfeatures: (30337, 2048) -> X_tr
# AWA2_train_attributelabel: (30337, 85)

#opts = parse_args()

# food 11 dataset:
train_data = np.load('../new_split/resnet101_trainfeatures.npy')
test_data = np.load('../new_split/resnet101_testfeatures.npy')
train_class_attributes_labels_continuous_allset = np.load('../new_split/Food11_train_Label_attributelabel.npy')
test_labels = np.load('../new_split/Food11_testlabel.npy')
test_classes_id_list = [0, 1, 3, 6, 8]
test_classes_id = np.asarray(test_classes_id_list)
test_class_attributes_labels_continuous = np.load('../new_split/S_te_gt_food_Label_normed.npy')

'''
# AWA2 continuous attributes:
train_data = np.load('../data/resnet101_trainfeatures.npy')
test_data = np.load('../data/resnet101_testfeatures.npy')
train_class_attributes_labels_continuous_allset = np.load('../data/AWA2_train_continuous_01_attributelabel.npy')
test_labels = np.load('../data/AWA2_testlabel.npy')
test_classes_id_list = [5, 13, 14, 17, 23, 24, 33, 38, 41, 47]
test_classes_id = np.asarray(test_classes_id_list)
test_class_attributes_labels_continuous = np.load('../data/S_te_gt.npy')
'''
'''
#AWA2 word vector labels:
train_data = np.load('../data/resnet101_trainfeatures.npy')
test_data = np.load('../data/resnet101_testfeatures.npy')
train_class_attributes_labels_continuous_allset = np.load('../data/AWA2_train_Label_attributelabel.npy')
test_labels = np.load('../data/AWA2_testlabel.npy')
test_classes_id_list = [5, 13, 14, 17, 23, 24, 33, 38, 41, 47]
test_classes_id = np.asarray(test_classes_id_list)
test_class_attributes_labels_continuous = np.load('../data/S_te_gt_Label.npy')
'''
'''
# AWA2 binary attributes:
train_data = np.load('../data/resnet101_trainfeatures.npy')
test_data = np.load('../data/resnet101_testfeatures.npy')
train_class_attributes_labels_continuous_allset = np.load('../data/AWA2_train_attributelabel.npy')
test_labels = np.load('../data/AWA2_testlabel.npy')
test_classes_id_list = [5, 13, 14, 17, 23, 24, 33, 38, 41, 47]
test_classes_id = np.asarray(test_classes_id_list)
test_class_attributes_labels_continuous = np.load('../data/bi_te_att_10_85.npy')
'''

##### Normalize the data
train_data = normalizeFeature(train_data.transpose()).transpose() 

##### Training
# SAE
W = SAE(train_data.transpose(), train_class_attributes_labels_continuous_allset.transpose(), ld) 

##### Test
    
    
# [F --> S], projecting data from feature space to semantic space: 84.68% for AwA dataset
semantic_predicted = np.dot(test_data, normalizeFeature(W).transpose())
[zsl_accuracy, y_hit_k] = zsl_acc(semantic_predicted, test_class_attributes_labels_continuous)
print('[1] zsl accuracy for AwA dataset [F >>> S]: {:.2f}%'.format(zsl_accuracy))

#attack_types = ["persian+cat", "hippopotamus", "leopard", "humpback+whale","seal","chimpanzee", "rat", "gaint+panda", "pig", "raccoon"]
#C2 = confusion_matrix(test_labels, y_hit_k, labels=[5, 13, 14, 17, 23, 24, 33, 38, 41, 47])


# [S --> F], projecting from semantic to visual space: 84.00% for AwA dataset
test_predicted = np.dot(normalizeFeature(test_class_attributes_labels_continuous.transpose()).transpose(), normalizeFeature(W))
[zsl_accuracy, y_hit_k] = zsl_acc(test_data, test_predicted)
print('[2] zsl accuracy for AwA dataset [S >>> F]: {:.2f}%'.format(zsl_accuracy))
'''
attack_types = ["bread", "dairy", "egg", "noodles-pasta", "seafood"]
C2 = confusion_matrix(np.array(test_labels), np.array(y_hit_k), labels=[0, 1, 3, 6, 8])
C2 = C2.astype('float') / C2.sum(axis=1)[:, np.newaxis]
C2 = np.around(C2, decimals=2)
df_cm = pd.DataFrame(C2, index=attack_types, columns=attack_types)
plt.figure(figsize=(10,7))
sns.set(font_scale=1.4) # for label size
sns.heatmap(df_cm, annot=True, annot_kws={"size": 16}, cmap="Blues") # font size

plt.show()
'''


(300, 2048)
[1] zsl accuracy for AwA dataset [F >>> S]: 38.71%
[2] zsl accuracy for AwA dataset [S >>> F]: 49.92%


'\nattack_types = ["bread", "dairy", "egg", "noodles-pasta", "seafood"]\nC2 = confusion_matrix(np.array(test_labels), np.array(y_hit_k), labels=[0, 1, 3, 6, 8])\nC2 = C2.astype(\'float\') / C2.sum(axis=1)[:, np.newaxis]\nC2 = np.around(C2, decimals=2)\ndf_cm = pd.DataFrame(C2, index=attack_types, columns=attack_types)\nplt.figure(figsize=(10,7))\nsns.set(font_scale=1.4) # for label size\nsns.heatmap(df_cm, annot=True, annot_kws={"size": 16}, cmap="Blues") # font size\n\nplt.show()\n'